# This notebook is used for the capstone week 3 project

## Project Part 1

In [54]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from tabulate import tabulate

In [55]:
source = requests.get('http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
table = pd.read_html(str(soup.table))
nh=pd.DataFrame(table[0])
nh.head(11)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


In [56]:
nh = nh[nh.Borough != 'Not assigned']
print('The shape of nh :{}'.format(nh.shape))
nh.head(10)

The shape of nh :(210, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [57]:
nh['Neighbourhood']=nh['Neighbourhood'].replace('Not assigned', nh['Borough'])
nh.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [58]:
df1 = nh.groupby(by=['Postcode','Borough']).agg(lambda x: ','.join(x))
df1.reset_index(level=['Postcode','Borough'], inplace=True)
df1.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [59]:
df1.shape

(103, 3)

## Project Part 2:

In [60]:
!conda install -c conda-forge geocoder --yes
#convert an address into latitude and longitude values
import geocoder

Solving environment: done

# All requested packages already installed.



In [61]:
df2 = df1
df2['Latitude'] = 0
df2['Longitude'] =0
df2.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",0,0
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",0,0
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",0,0
3,M1G,Scarborough,Woburn,0,0
4,M1H,Scarborough,Cedarbrae,0,0
5,M1J,Scarborough,Scarborough Village,0,0
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",0,0
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",0,0
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",0,0
9,M1N,Scarborough,"Birch Cliff,Cliffside West",0,0


In [62]:
for i in range(df2.shape[0]):
    # initialize your variable to None
    coords = None
    # loop until you get the coordinates
    while(coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(df2.iloc[i,0]))
        coords = g.latlng
        df2.iloc[i,3] = coords[0]
        df2.iloc[i,4] = coords[1]

df2.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.726276,-79.263625
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.713054,-79.285055
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.724235,-79.227925
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.696770,-79.259967


## Project Part 3:

In [97]:
import numpy as np # library to handle data in a vectorized manner
!conda install -c conda-forge geopy --yes # uncomment this line if you haven'tcompleted the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#from geocoder import Nominatim
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



## Step 1

In [98]:
df3 = df2
print('The dataframe has {} boroughs and {} Postcode Neighborhoods.'.format(
len(df3['Borough'].unique()),
df3.shape[0]
)
)

The dataframe has 11 boroughs and 103 Postcode Neighborhoods.


In [99]:
DT_df3 = df3[df3['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
print('DT_df3 shape ( {}, {} )'.format(DT_df3.shape[0],DT_df3.shape[1]))
DT_df3.head(18)

DT_df3 shape ( 19, 5 )


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.682205,-79.377945
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.668160,-79.366602
2,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302
3,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657363,-79.378180
5,M5C,Downtown Toronto,St. James Town,43.651210,-79.375481
6,M5E,Downtown Toronto,Berczy Park,43.645160,-79.373675
7,M5G,Downtown Toronto,Central Bay Street,43.656091,-79.384930
8,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.649700,-79.382582
9,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.630210,-79.362433


## Step 2: Create a map of Toronto,ON with neighborhoods superimposed ontop.

In [100]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.653963, -79.387207.


In [101]:
# create map of Toronto, ON using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)

map_toronto

##### Now visualization Downtown Toronto the neighborhoods in it.

In [102]:
address = 'Downtown Toronto, ON'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [103]:
# create map of Downtown Toronto, ON using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=13)
# add markers to map
for lat, lng, borough, neighborhood in zip(DT_df3['Latitude'], DT_df3['Longitude'], DT_df3['Borough'], DT_df3['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_downtown)

map_downtown

In [104]:
CLIENT_ID = 'VBJRAO5IMFVHJ15014BHKY1WXSLLA1JVQ0L0HQGMAIF4PQLH' # your Foursquare ID
CLIENT_SECRET = 'GFNHX055SNPUJUVPMUENX3JQKZ5MGXQ5TWQKBHEROMXTR5L3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Now, let's get the top 100 venues that are in Downtown Toronto within a radius of 1200 meters.

In [105]:
neighborhood_latitude = DT_df3.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = DT_df3.loc[0, 'Longitude'] # neighborhood longitude value
LIMIT = 100
radius = 1200
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
neighborhood_latitude,
neighborhood_longitude,
radius,
LIMIT)
#url
results= requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e476433d03993001b662bfa'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 61,
  'suggestedBounds': {'ne': {'lat': 43.69300501080008,
    'lng': -79.36303905909658},
   'sw': {'lat': 43.671404989200056, 'lng': -79.39285133490337}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4adcb343f964a520e32e21e3',
       'name': 'Summerhill Market',
       'location': {'address': '446 Summerhill Ave',
        'crossStreet': 'btwn. MacLennan Ave. and Glen Rd.',
        'lat': 43.68626482142425,
        'lng': -79.37545823237794,
      

##### Now we are ready to clean the json and structure it into a pandas dataframe.

In [106]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [107]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head(10)

,name,categories,lat,lng
0,Summerhill Market,Grocery Store,43.686265,-79.375458
1,Evergreen Brick Works Farmers Market,Farmers Market,43.684282,-79.365649
2,Toronto Lawn Tennis Club,Athletics & Sports,43.680667,-79.388559
3,Chorley Park,Park,43.687163,-79.370538
4,Evergreen Brick Works,Historic Site,43.684401,-79.365242
5,LCBO,Liquor Store,43.681497,-79.391261
6,Greenhouse Juice Co,Juice Bar,43.679101,-79.390686
7,Boxcar Social,Coffee Shop,43.681750,-79.391447
8,Black Camel,BBQ Joint,43.677016,-79.389367
9,The Don Valley Brick Works Park,Park,43.685051,-79.367015


In [108]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

61 venues were returned by Foursquare.


##### Create a function to repeat the same process to all the neighborhoods in Downtown Toronto,ON

In [109]:
def getNearbyVenues(postcodes, latitudes, longitudes, radius=1000):
    venues_list=[]
    for postcode, lat, lng in zip(postcodes, latitudes, longitudes):
        #print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius,
        LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
        postcode,
        lat,
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'],
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in results])
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Postcode',
        'Neighborhood Latitude',
        'Neighborhood Longitude',
        'Venue',
        'Venue Latitude',
        'Venue Longitude',
        'Venue Category']
    return(nearby_venues)

In [110]:
Downtown_venues = getNearbyVenues(postcodes=DT_df3['Postcode'],
latitudes=DT_df3['Latitude'],
longitudes=DT_df3['Longitude']
)

In [111]:
print(Downtown_venues.shape)
Downtown_venues.head(10)

(1662, 7)


,Postcode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4W,43.682205,-79.377945,Summerhill Market,43.686265,-79.375458,Grocery Store
1,M4W,43.682205,-79.377945,Chorley Park,43.687163,-79.370538,Park
2,M4W,43.682205,-79.377945,Toronto Lawn Tennis Club,43.680667,-79.388559,Athletics & Sports
3,M4W,43.682205,-79.377945,The Don Valley Brick Works Park,43.685051,-79.367015,Park
4,M4W,43.682205,-79.377945,Rosehill Reservoir,43.686304,-79.388133,Park
5,M4W,43.682205,-79.377945,David A. Balfour Park,43.685629,-79.388078,Park
6,M4W,43.682205,-79.377945,Craigleigh Gardens,43.678099,-79.371586,Park
7,M4W,43.682205,-79.377945,Summerhill,43.682976,-79.389123,Neighborhood
8,M4W,43.682205,-79.377945,The Beer Store,43.680217,-79.390074,Beer Store
9,M4W,43.682205,-79.377945,Rosedale Park,43.682328,-79.378934,Playground


##### Let's check how many venues were returned for each poscode

In [112]:
Downtown_venues.groupby('Postcode').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postcode,,,,,,
M4W,16,16,16,16,16,16
M4X,38,38,38,38,38,38
M4Y,100,100,100,100,100,100
M5A,97,97,97,97,97,97
M5B,100,100,100,100,100,100
M5C,100,100,100,100,100,100
M5E,100,100,100,100,100,100
M5G,100,100,100,100,100,100
M5H,100,100,100,100,100,100


In [113]:
print('There are {} uniques categories.'.format(len(Downtown_venues['Venue Category'].unique())))

There are 209 uniques categories.


### Step 3, Analyze each zopcode neiboughhood

In [114]:
# one hot encoding
Downtown_onehot = pd.get_dummies(Downtown_venues[['Venue Category']], prefix="",prefix_sep="")
# add neighborhood column back to dataframe
Downtown_onehot['Postcode'] = Downtown_venues['Postcode']
# move neighborhood column to the first column
fixed_columns = [Downtown_onehot.columns[-1]] + list(Downtown_onehot.columns[:-1])
Downtown_onehot = Downtown_onehot[fixed_columns]
Downtown_onehot.head(10)

,Postcode,Afghan Restaurant,American Restaurant,Animal Shelter,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4W,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M4W,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M4W,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [115]:
Downtown_onehot.shape

(1662, 210)

In [116]:
Downtown_grouped = Downtown_onehot.groupby('Postcode').mean().reset_index()
Downtown_grouped

,Postcode,Afghan Restaurant,American Restaurant,Animal Shelter,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4W,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.062500,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.062500,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.0625,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.062500,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.0625,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.125,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.437500,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.0625,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,

In [117]:
Downtown_grouped.shape

(19, 210)

In [118]:
num_top_venues = 10
for hood in Downtown_grouped['Postcode']:
    print("----"+hood+"----")
    temp = Downtown_grouped[Downtown_grouped['Postcode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4W----
                     venue  freq
0                     Park  0.44
1             Neighborhood  0.12
2               Beer Store  0.06
3              Candy Store  0.06
4               Playground  0.06
5       Athletics & Sports  0.06
6                    Trail  0.06
7           Mattress Store  0.06
8            Grocery Store  0.06
9  New American Restaurant  0.00


----M4X----
                  venue  freq
0                  Park  0.08
1            Restaurant  0.05
2                  Pool  0.05
3                  Café  0.05
4   Japanese Restaurant  0.05
5             Gastropub  0.05
6                 Diner  0.05
7  Caribbean Restaurant  0.03
8           Coffee Shop  0.03
9          Dance Studio  0.03


----M4Y----
                  venue  freq
0           Coffee Shop  0.09
1   Japanese Restaurant  0.05
2                  Park  0.04
3               Gay Bar  0.03
4                  Café  0.03
5           Men's Store  0.03
6                   Gym  0.03
7                 Diner  0.

##### Put that into a pandas dataframe

First, write a function to sort the venues in descending order.

In [119]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

Second create the new dataframe and display the top 5 venues for each neighborhood.

In [120]:
num_top_venues = 8
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# create a new dataframe
Postcodes_venues_sorted = pd.DataFrame(columns=columns)
Postcodes_venues_sorted['Postcode'] = Downtown_grouped['Postcode']
for ind in np.arange(Downtown_grouped.shape[0]):
    Postcodes_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Downtown_grouped.iloc[ind, :], num_top_venues)
    
Postcodes_venues_sorted.head(10)

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,M4W,Park,Neighborhood,Playground,Trail,Beer Store,Grocery Store,Athletics & Sports,Mattress Store
1,M4X,Park,Café,Japanese Restaurant,Diner,Pool,Restaurant,Gastropub,Steakhouse
2,M4Y,Coffee Shop,Japanese Restaurant,Park,Gay Bar,Men's Store,Café,Gym,Caribbean Restaurant
3,M5A,Coffee Shop,Café,Pub,Italian Restaurant,Theater,Park,Bakery,Boat or Ferry
4,M5B,Coffee Shop,Clothing Store,Café,Italian Restaurant,Diner,Gastropub,Restaurant,Japanese Restaurant
5,M5C,Café,Coffee Shop,Restaurant,Hotel,Bakery,Cosmetics Shop,Seafood Restaurant,Breakfast Spot
6,M5E,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Beer Bar,Bakery,Park
7,M5G,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Italian Restaurant,Cosmetics Shop,Chinese Restaurant,Bookstore
8,M5H,Café,Coffee Shop,Hotel,Theater,Restaurant,Italian Restaurant,Steakhouse,Japanese Restaurant
9,M5J,Harbor / Marina,Beach,Café,Pier,Disc Golf,Boat or Ferry,Park,Yoga Studio


### Step 4, Cluster neighborhoods

##### Run k-means to cluster the Borough into 5 clusters.

In [121]:
# set number of clusters
kclusters = 5
Downtown_grouped_clustering = Downtown_grouped.drop('Postcode', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Downtown_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 4, 4, 4, 0, 0, 4, 0, 2], dtype=int32)

##### Now create a new dataframe that includes the cluster as well as the top 10 venues for each postcode neighborhood.

In [122]:
Postcodes_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Downtown_merged = DT_df3
# merge Downtown_grouped with DT_df3 to add latitude/longitude for each postcode borough
Downtown_merged = Downtown_merged.join(Postcodes_venues_sorted.set_index('Postcode'), on='Postcode')
print(Downtown_merged.shape)
Downtown_merged.head(10) # check the last columns!

(19, 14)


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.682205,-79.377945,1,Park,Neighborhood,Playground,Trail,Beer Store,Grocery Store,Athletics & Sports,Mattress Store
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.668160,-79.366602,0,Park,Café,Japanese Restaurant,Diner,Pool,Restaurant,Gastropub,Steakhouse
2,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302,4,Coffee Shop,Japanese Restaurant,Park,Gay Bar,Men's Store,Café,Gym,Caribbean Restaurant
3,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166,4,Coffee Shop,Café,Pub,Italian Restaurant,Theater,Park,Bakery,Boat or Ferry
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657363,-79.378180,4,Coffee Shop,Clothing Store,Café,Italian Restaurant,Diner,Gastropub,Restaurant,Japanese Restaurant
5,M5C,Downtown Toronto,St. James Town,43.651210,-79.375481,0,Café,Coffee Shop,Restaurant,Hotel,Bakery,Cosmetics Shop,Seafood Restaurant,Breakfast Spot
6,M5E,Downtown Toronto,Berczy Park,43.645160,-79.373675,0,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Beer Bar,Bakery,Park
7,M5G,Downtown Toronto,Central Bay Street,43.656091,-79.384930,4,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Italian Restaurant,Cosmetics Shop,Chinese Restaurant,Bookstore
8,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.649700,-79.382582,0,Café,Coffee Shop,Hotel,Theater,Restaurant,Italian Restaurant,Steakhouse,Japanese Restaurant
9,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.630210,-79.362433,2,Harbor / Marina,Beach,Café,Pier,Disc Golf,Boat or Ferry,Park,Yoga Studio


Finally, visualize the resulting clusters

In [123]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Downtown_merged['Latitude'], Downtown_merged['Longitude'], Downtown_merged['Postcode'], Downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Step 5. Examine Clusters

#### Cluster 1

In [124]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 0, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
1,Downtown Toronto,0,Park,Café,Japanese Restaurant,Diner,Pool,Restaurant,Gastropub,Steakhouse
5,Downtown Toronto,0,Café,Coffee Shop,Restaurant,Hotel,Bakery,Cosmetics Shop,Seafood Restaurant,Breakfast Spot
6,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Beer Bar,Bakery,Park
8,Downtown Toronto,0,Café,Coffee Shop,Hotel,Theater,Restaurant,Italian Restaurant,Steakhouse,Japanese Restaurant
10,Downtown Toronto,0,Café,Coffee Shop,Hotel,Steakhouse,Restaurant,Italian Restaurant,Theater,Seafood Restaurant
11,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Concert Hall,Steakhouse,Gastropub,Beer Bar
15,Downtown Toronto,0,Coffee Shop,Hotel,Café,Pizza Place,Bar,Gym,Gastropub,Steakhouse
16,Downtown Toronto,0,Café,Hotel,Coffee Shop,Restaurant,Italian Restaurant,Japanese Restaurant,Steakhouse,Theater


#### Cluster 2

In [125]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 1, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,Downtown Toronto,1,Park,Neighborhood,Playground,Trail,Beer Store,Grocery Store,Athletics & Sports,Mattress Store


#### Cluster 3

In [126]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 2, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
9,Downtown Toronto,2,Harbor / Marina,Beach,Café,Pier,Disc Golf,Boat or Ferry,Park,Yoga Studio


#### Cluster 4

In [127]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 3, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
12,Downtown Toronto,3,Café,Bakery,Restaurant,Bar,Vegetarian / Vegan Restaurant,Bookstore,Coffee Shop,Mexican Restaurant
13,Downtown Toronto,3,Café,Bar,Coffee Shop,Vegetarian / Vegan Restaurant,Yoga Studio,Ice Cream Shop,Mexican Restaurant,Dessert Shop
17,Downtown Toronto,3,Korean Restaurant,Café,Grocery Store,Coffee Shop,Cocktail Bar,Japanese Restaurant,Mexican Restaurant,Indian Restaurant


#### Cluster 5

In [128]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 4, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
2,Downtown Toronto,4,Coffee Shop,Japanese Restaurant,Park,Gay Bar,Men's Store,Café,Gym,Caribbean Restaurant
3,Downtown Toronto,4,Coffee Shop,Café,Pub,Italian Restaurant,Theater,Park,Bakery,Boat or Ferry
4,Downtown Toronto,4,Coffee Shop,Clothing Store,Café,Italian Restaurant,Diner,Gastropub,Restaurant,Japanese Restaurant
7,Downtown Toronto,4,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Italian Restaurant,Cosmetics Shop,Chinese Restaurant,Bookstore
14,Downtown Toronto,4,Coffee Shop,Bar,Yoga Studio,Gym,Italian Restaurant,Bakery,Park,Café
18,Downtown Toronto,4,Coffee Shop,Gastropub,Japanese Restaurant,Bubble Tea Shop,Thai Restaurant,Italian Restaurant,Park,Sushi Restaurant
